In [ ]:
import os
os.chdir("../")


In [ ]:
%config Completer.use_jedi = False



In [ ]:
import yaml
import torch
import random
import argparse
import numpy as np
import matplotlib
import pandas as pd
from src.dataset_hrg import getDataLoader
from src.module_hrg import EmbeddingHRG
from torch_geometric.data.batch import Batch
from torch.optim import AdamW
import os
%load_ext autoreload
%autoreload 2


In [ ]:
## Classifier
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import numpy as np
import torch
import torch.nn as nn

class DurationClassifier(torch.nn.Module):
    def __init__(self, h_dim, n_buckets):
        super(DurationClassifier, self).__init__()
        self.h_dim = h_dim
        self.gcn = EmbeddingHRG(data_tr.dataset.n_vocab, self.h_dim, self.h_dim).cuda()
        self.n_buckets = n_buckets
        self.h_dim = h_dim
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.criterion = nn.CrossEntropyLoss(ignore_index=-1)
        self.cls_head = nn.Sequential(nn.Linear(self.h_dim, self.h_dim // 2),
                                      nn.ReLU(),
                                      nn.Linear(self.h_dim // 2, self.n_buckets))
    def forward(self, x, durations):
        x = self.gcn(x)
        bsz, max_syll_nodes, h_dim = x.shape
        duration_labels = self.get_duration_label_tensor(durations, max_syll_nodes)
        logits = self.cls_head(x)
        loss = self.criterion(logits.view(-1, self.n_buckets), duration_labels.view(-1))
        return loss, logits
    
    def get_duration_label_tensor(self, durations, max_syll_nodes):
        syll_dur = torch.zeros(len(durations), max_syll_nodes).long() - 1
        for i in range(len(durations)):
            syll_dur[i, :len(durations[i])] = torch.LongTensor(durations[i])
        return syll_dur.to(self.device)

        
        

### Step 1: Load dataset


In [ ]:
config = yaml.load(open("config/config-arctic.yaml", 'r'), Loader=yaml.FullLoader)
_config = config['solver']

In [ ]:
data_tr = getDataLoader(
    mode='train',
    meta_path=_config['meta_path']['train'],
    data_dir=_config['data_dir'],
    batch_size=_config['batch_size'],
    r=config['model']['tacotron']['r'],
    n_jobs=_config['n_jobs'],
    use_gpu=True,
    add_info_headers=["dur"])

In [ ]:
n_buckets = 11
h_dim = 256
label_dict = {f"DB_{i}": i for i in range(n_buckets)}

### Step 2: Init the model

In [ ]:
dur_classifier = DurationClassifier(h_dim=h_dim, n_buckets=n_buckets).cuda()

In [ ]:
optim = AdamW(dur_classifier.parameters(), lr=0.001)

In [ ]:
def parse_duration_sequence(label_seq):
    label_seq = [ls.strip().split() for ls in label_seq]
    res = []
    for ls in label_seq:
        res.append([label_dict[x] for x in ls])
    return res
    

### Training loop

In [ ]:
for epoch in range(5):
    running_loss = 0.0
    for i, batch in enumerate(data_tr, 0):
        _, txt, text_lengths, mel, spec, durations = batch
        durations = parse_duration_sequence(durations)
        
        optim.zero_grad()

        loss, logits = dur_classifier(txt, durations)
        loss.backward()
        optim.step()

        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
data_test = getDataLoader(
    mode='test',
    meta_path=_config['meta_path']['test'],
    data_dir=_config['data_dir'],
    batch_size=_config['batch_size'],
    r=config['model']['tacotron']['r'],
    n_jobs=_config['n_jobs'],
    use_gpu=True,
    vocab=data_tr.dataset.vocab,
    add_info_headers=["dur"])

### Inference

In [ ]:
dur_classifier = dur_classifier.eval()
with torch.no_grad():
    for i, batch in enumerate(data_test, 0):
        _, txt, text_lengths, mel, spec, durations = batch
        durations = parse_duration_sequence(durations)
        loss, logits = dur_classifier(txt, durations)
        break
dur_classifier = dur_classifier.train()

In [ ]:
def get_duration_accuracy(self, durations_true, logits):
    durations_predicted = logits.argmax(-1)
    durations_predicted = durations_predicted.detach().cpu().numpy()
    num, den = 0., 0.
    for i, d in enumerate(durations_predicted):
        num += (durations_predicted[i, :len(durations_true[i])] == durations_true[i]).sum()
        den += len(durations_true[i])
    return num / den

In [ ]:
get_duration_accuracy(durations_true=durations, durations_predicted=logits.argmax(-1))

In [ ]:
durations_predicted=logits.argmax(-1)

In [ ]:
durations_predicted = durations_predicted.detach().cpu().numpy()

In [ ]:
durations_predicted[0, :len(durations[0])] == np.array(durations[0])

In [ ]:
pad_durations()

In [ ]:
durations

In [ ]:
durations = pad_durations(durations)

In [ ]:
durations.shape

In [ ]:
log

In [ ]:
class_output.shape

### Add duration

In [ ]:
meta_all = pd.read_csv("arctic/meta_all.txt", sep="|", names=["mel", "lin", "len", "hrg", "add_info"])
meta_all = meta_all.drop(labels="add_info", axis=1)
syll_dur = pd.read_csv("syll_dur_end.txt", sep="|", names=["source", "target", "end"])
merged = pd.concat([meta_all, syll_dur], axis=1)
merged[["mel", "lin", "len", "hrg", "target"]].to_csv("arctic/meta_all.txt", sep="|", header=False, index=None)